In [ ]:
%matplotlib inline



# Exchange Data Between Servers
In this example, 2 dpf's servers will be started and a workflow will be
created with a part on both servers. This example opens the possibility for a
user to read data from a given machine and transform this data on another
without any more difficulties than working on a local computer


In [ ]:
from ansys.dpf import core as dpf
from ansys.dpf.core import examples
from ansys.dpf.core import operators as ops
from ansys.jupyterhub.manager import spawn_dpf, delete_pod

# Create 2 servers
Start two dpf servers.



In [ ]:
server1, pod1_name = spawn_dpf()
server2, pod2_name = spawn_dpf()
servers = [server1, server2]

# Send the result file
Here, the result file is sent in a temporary dir of the first server.



In [ ]:
file = examples.complex_rst
file_path_in_tmp = dpf.upload_file_in_tmp_folder(file, server=server1)

# Create a workflow on the first server
Create the model



In [ ]:
model = dpf.Model(file_path_in_tmp, server=server1)

# Read displacement
disp = model.results.displacement()
disp.inputs.time_scoping(len(model.metadata.time_freq_support.time_frequencies))

# Create a workflow on the second server



In [ ]:
# Change the cartesian coordinates to cylindrical coordinates cs
coordinates = ops.geo.rotate_in_cylindrical_cs_fc(server=server2)

# Create the cartesian coordinate cs
cs = dpf.fields_factory.create_scalar_field(12, server=server2)
cs.data = [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]

coordinates.inputs.coordinate_system(cs)

# choose the radial component to plot
comp = dpf.operators.logic.component_selector_fc(coordinates, 0, server=server2)

# Pass data from on server to another



In [ ]:
fc_disp = disp.outputs.fields_container()
fc_copy = fc_disp.deep_copy(server=server2)

mesh_copy = model.metadata.meshed_region.deep_copy(server=server2)
# give a mesh to the field
fc_copy[0].meshed_region = mesh_copy
fc_copy[1].meshed_region = mesh_copy

coordinates.inputs.field(fc_copy)

# Plot the output



In [ ]:
out = comp.outputs.fields_container()

# real part
mesh_copy.plot(out.get_field({"complex": 0}))

# imaginary part
mesh_copy.plot(out.get_field({"complex": 1}))

In [ ]:
delete_pod(pod1_name)
delete_pod(pod2_name)